In [7]:
%pylab inline

import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [8]:
 pd.set_option('display.notebook_repr_html', False)

* IPython notebook is one global namespace, use a class to keep namespace cleaner
* few globals are used here:
    * X_all
    * RADIUS

# events within a "radius"

* given a series with timestamps of events
* find for all members separately all the events that are within a radius
* foreach member clean the events found within the radius

### test data

In [9]:
class EventStream(object):
    data = {'epochtime': [# 1
                          # 4 events within 61 sec
                          1438797641, 1438807651,
                          1438907663, 1438907633, 1438907693, 1438907694,
                          1439797999,
                          # 2
                          # 4 events within 62 sec
                          # events are far apart from any other member
                          1438797651, 1538807651,
                          1538907663, 1538907633, 1538907693, 1538907695,
                          1539797999,
                          # 3
                          # 4 events within 60 sec
                          # events are only 1 second apart from member 1
                          1438797644, 1438807654,
                          1438907664, 1438907634, 1438907694, 1438907694,
                          1439797994,
                          # 4
                          # 1 event in the "neighborhood of center point"
                          # different member though
                          1438907665,
                         ],
            'event': ['up', 'down', 'down', 'down', 'down', 'up', 'up',
                      'up', 'down', 'up', 'up', 'down', 'up', 'down',
                      'up', 'up', 'down', 'down', 'up', 'up', 'down',
                      'up',
                     ],
            'member_id': [1]*7 + [2]*7 + [3]*7 + [4]}

X_all = pd.DataFrame(EventStream.data)
X_all['offset'] = 0
X_all['counter'] = 1  # not strictly needed
X_all

     epochtime event  member_id  offset  counter
0   1438797641    up          1       0        1
1   1438807651  down          1       0        1
2   1438907663  down          1       0        1
3   1438907633  down          1       0        1
4   1438907693  down          1       0        1
5   1438907694    up          1       0        1
6   1439797999    up          1       0        1
7   1438797651    up          2       0        1
8   1538807651  down          2       0        1
9   1538907663    up          2       0        1
10  1538907633    up          2       0        1
11  1538907693  down          2       0        1
12  1538907695    up          2       0        1
13  1539797999  down          2       0        1
14  1438797644    up          3       0        1
15  1438807654    up          3       0        1
16  1438907664  down          3       0        1
17  1438907634  down          3       0        1
18  1438907694    up          3       0        1
19  1438907694    up

## separate members from each other

### groupby and iterate

* one solution could be to groupby members
* process each member individually

In [10]:
class GroupIterate(object):
    members = None
    ix = None
    X = None
gi = GroupIterate()

gi.members = X_all.groupby('member_id')
gi.ix = gi.members.groups.itervalues().next()
gi.ix

[0, 1, 2, 3, 4, 5, 6]

In [11]:
gi.X = X_all.iloc[gi.ix][['offset', 'epochtime']]
gi.X

   offset   epochtime
0       0  1438797641
1       0  1438807651
2       0  1438907663
3       0  1438907633
4       0  1438907693
5       0  1438907694
6       0  1439797999

### separate members with an offset vector
* look at each member individually
* find values that are within a given radius for a specific member
* values of different members are never within a given radius

In [12]:
import sklearn.neighbors as neighbors

class OffsetVector(object):
    RADIUS = 3
    data = None
    df = None
    members = None
    idx = None
    distance = None

ov = OffsetVector()

In [13]:
ov.data = {'score': [1,2,9,
                  2,3,12],
        'member': [1]*3 + [2]*3
       }

ov.df = pd.DataFrame(ov.data)

add autoincrement idx to member

In [14]:
ov.members = pd.DataFrame(ov.df['member'].unique(), columns=['member'])
ov.members['idx'] = ov.members.index
ov.members

   member  idx
0       1    0
1       2    1

In [15]:
ov.idx = pd.merge(ov.df, ov.members)
ov.idx

   member  score  idx
0       1      1    0
1       1      2    0
2       1      9    0
3       2      2    1
4       2      3    1
5       2     12    1

offset members by RADIUS + 1

In [16]:
ov.idx['offset'] = ov.idx['idx'] * ov.RADIUS + ov.idx['idx'] - 1
ov.idx

   member  score  idx  offset
0       1      1    0      -1
1       1      2    0      -1
2       1      9    0      -1
3       2      2    1       3
4       2      3    1       3
5       2     12    1       3

In [17]:
ov.distance = neighbors.DistanceMetric.get_metric('manhattan')

ov.distance.pairwise(ov.idx[['score', 'offset']])

array([[  0.,   1.,   8.,   5.,   6.,  15.],
       [  1.,   0.,   7.,   4.,   5.,  14.],
       [  8.,   7.,   0.,  11.,  10.,   7.],
       [  5.,   4.,  11.,   0.,   1.,  10.],
       [  6.,   5.,  10.,   1.,   0.,   9.],
       [ 15.,  14.,   7.,  10.,   9.,   0.]])

## find events within a radius

In [18]:
RADIUS = 60

### nearest neighbors

* use a nearest neighbor algorithm
* distance function of neighbor algorithm needs (x,y)
* insert x=offset column

In [19]:
import sklearn.neighbors as neighbors

class NearestNeighbor(object):
    members = None
    X = None

nn = NearestNeighbor()

In [20]:
nn.members = pd.DataFrame(X_all['member_id'].unique(), columns=['member_id'])
nn.members['member_idx'] = nn.members.index

X_all = pd.merge(X_all, nn.members)

X_all['offset'] = X_all['member_idx'] * RADIUS + X_all['member_idx'] - 1
X_all

     epochtime event  member_id  offset  counter  member_idx
0   1438797641    up          1      -1        1           0
1   1438807651  down          1      -1        1           0
2   1438907663  down          1      -1        1           0
3   1438907633  down          1      -1        1           0
4   1438907693  down          1      -1        1           0
5   1438907694    up          1      -1        1           0
6   1439797999    up          1      -1        1           0
7   1438797651    up          2      60        1           1
8   1538807651  down          2      60        1           1
9   1538907663    up          2      60        1           1
10  1538907633    up          2      60        1           1
11  1538907693  down          2      60        1           1
12  1538907695    up          2      60        1           1
13  1539797999  down          2      60        1           1
14  1438797644    up          3     121        1           2
15  1438807654    up    

In [21]:
nn.X = X_all[['epochtime', 'offset']]

* create the k-d tree

In [22]:
class NNTree(object):
    kd = None
    dist = None
    ix = None
    events_in_radius = None
tree = NNTree()

tree.kd = neighbors.KDTree(nn.X, leaf_size=1, metric='manhattan')
tree.kd

* take a look at the distances
* what is the nearest neighbor to event at pos=0?

In [23]:
tree.dist, tree.ix = tree.kd.query(nn.X.iloc[0], k=2)
tree.dist, tree.ix

(array([[  0.,  71.]]), array([[0, 7]]))

* find the timestamps that are within a given radius
* value of radius depends on distance function

In [24]:
tree.events_in_radius = tree.kd.query_radius(nn.X, r=RADIUS)
tree.events_in_radius

array([array([0]), array([1]), array([3, 2, 4, 5]), array([3, 2, 4]),
       array([3, 2, 4, 5]), array([2, 4, 5]), array([6]), array([7]),
       array([8]), array([10,  9, 11, 12]), array([10,  9, 11]),
       array([10,  9, 11, 12]), array([ 9, 11, 12]), array([13]),
       array([14]), array([15]), array([16, 17, 19, 18]),
       array([16, 17, 19, 18]), array([16, 17, 19, 18]),
       array([16, 17, 19, 18]), array([20]), array([21])], dtype=object)

#### extract clusters
* only keep the supersets

In [25]:
class NNCluster(object):
    in_radius = None
    supersets = None
    only_supersets = None
    min_sets = None
    raw_ix = None
    neighbor_ix = None
    neighbor_mask = None
    multi_events = None

nnc = NNCluster()

In [26]:
nnc.in_radius = [set(e) for e in tree.events_in_radius]

nnc.supersets = np.vectorize(lambda x: not any([x.issubset(e) for e in nnc.in_radius if not e == x]))
nnc.only_supersets = nnc.supersets(nnc.in_radius)
nnc.only_supersets

array([ True,  True,  True, False,  True, False,  True,  True,  True,
        True, False,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True], dtype=bool)

In [27]:
nnc.min_sets = tree.events_in_radius[nnc.only_supersets]
nnc.min_sets

array([array([0]), array([1]), array([3, 2, 4, 5]), array([3, 2, 4, 5]),
       array([6]), array([7]), array([8]), array([10,  9, 11, 12]),
       array([10,  9, 11, 12]), array([13]), array([14]), array([15]),
       array([16, 17, 19, 18]), array([16, 17, 19, 18]),
       array([16, 17, 19, 18]), array([16, 17, 19, 18]), array([20]),
       array([21])], dtype=object)

* only interested in the nnc with more than 1 element

In [28]:
is_multi = np.vectorize(lambda x: len(x) > 1)
nnc.raw_ix = nnc.min_sets[is_multi(nnc.min_sets)]
nnc.raw_ix

array([array([3, 2, 4, 5]), array([3, 2, 4, 5]), array([10,  9, 11, 12]),
       array([10,  9, 11, 12]), array([16, 17, 19, 18]),
       array([16, 17, 19, 18]), array([16, 17, 19, 18]),
       array([16, 17, 19, 18])], dtype=object)

* only keep unique rows
* too narrow, can be less strict: only keep unique indexes
    * actually not correct: have to keep the "cluster id"
    * "cluster id" is required to only remove redundant payment per "cluster"
    * one member can have many "nnc"

In [29]:
np.vstack({tuple(row) for row in nnc.raw_ix})

array([[10,  9, 11, 12],
       [16, 17, 19, 18],
       [ 3,  2,  4,  5]])

* raw_ix is not a multi-dim ndarray
* default flatten does not work

In [30]:
nnc.raw_ix.reshape(-1)

array([array([3, 2, 4, 5]), array([3, 2, 4, 5]), array([10,  9, 11, 12]),
       array([10,  9, 11, 12]), array([16, 17, 19, 18]),
       array([16, 17, 19, 18]), array([16, 17, 19, 18]),
       array([16, 17, 19, 18])], dtype=object)

In [31]:
nnc.raw_ix.shape

(8,)

* flatten a numpy array of dtype object
* "cluster id" is lost in this step
    * for member with more than one "cluster" events are lost

In [32]:
np.hstack(nnc.raw_ix.flat)

array([ 3,  2,  4,  5,  3,  2,  4,  5, 10,  9, 11, 12, 10,  9, 11, 12, 16,
       17, 19, 18, 16, 17, 19, 18, 16, 17, 19, 18, 16, 17, 19, 18])

In [33]:
nnc.neighbor_ix = np.unique(np.hstack(nnc.raw_ix.flat))
nnc.neighbor_ix

array([ 2,  3,  4,  5,  9, 10, 11, 12, 16, 17, 18, 19])

In [34]:
nnc.neighbor_mask = X_all.index.isin(nnc.neighbor_ix)
nnc.neighbor_mask

array([False, False,  True,  True,  True,  True, False, False, False,
        True,  True,  True,  True, False, False, False,  True,  True,
        True,  True, False, False], dtype=bool)

### read data within neighborhood

* read the multi events from X using the indexes of the "nnc"

In [35]:
nnc.multi_events = X_all.iloc[nnc.neighbor_ix]
nnc.multi_events

     epochtime event  member_id  offset  counter  member_idx
2   1438907663  down          1      -1        1           0
3   1438907633  down          1      -1        1           0
4   1438907693  down          1      -1        1           0
5   1438907694    up          1      -1        1           0
9   1538907663    up          2      60        1           1
10  1538907633    up          2      60        1           1
11  1538907693  down          2      60        1           1
12  1538907695    up          2      60        1           1
16  1438907664  down          3     121        1           2
17  1438907634  down          3     121        1           2
18  1438907694    up          3     121        1           2
19  1438907694    up          3     121        1           2

### clustering

In [36]:
import sklearn.cluster as cluster

class SpatialCluster(object):
    X = None
    cluster = None
    neighbor_ix = None
    neighbor_mask = None
    multi_events = None
    
sc = SpatialCluster()

In [37]:
sc.X = X_all[['epochtime', 'offset']]
sc.X

     epochtime  offset
0   1438797641      -1
1   1438807651      -1
2   1438907663      -1
3   1438907633      -1
4   1438907693      -1
5   1438907694      -1
6   1439797999      -1
7   1438797651      60
8   1538807651      60
9   1538907663      60
10  1538907633      60
11  1538907693      60
12  1538907695      60
13  1539797999      60
14  1438797644     121
15  1438807654     121
16  1438907664     121
17  1438907634     121
18  1438907694     121
19  1438907694     121
20  1439797994     121
21  1438907665     182

In [38]:
sc.X.epochtime - sc.X.epochtime.min()

0             0
1         10010
2        110022
3        109992
4        110052
5        110053
6       1000358
7            10
8     100010010
9     100110022
10    100109992
11    100110052
12    100110054
13    101000358
14            3
15        10013
16       110023
17       109993
18       110053
19       110053
20      1000353
21       110024
Name: epochtime, dtype: int64

In [39]:
sc.cluster = cluster.DBSCAN(eps=RADIUS, min_samples=2, metric='manhattan', leaf_size=2)
sc.cluster.fit(np.array(sc.X, dtype=float))

DBSCAN(algorithm='auto', eps=60, leaf_size=2, metric='manhattan',
    min_samples=2, p=None, random_state=None)

In [40]:
sc.cluster.labels_

array([-1, -1,  2,  2,  2,  2, -1, -1, -1,  1,  1,  1,  1, -1, -1, -1,  0,
        0,  0,  0, -1, -1])

In [41]:
sc.neighbor_mask = np.zeros_like(sc.cluster.labels_, dtype=bool)
sc.neighbor_mask[sc.cluster.core_sample_indices_] = True
sc.neighbor_mask

array([False, False,  True,  True,  True,  True, False, False, False,
        True,  True,  True,  True, False, False, False,  True,  True,
        True,  True, False, False], dtype=bool)

In [42]:
sc.neighbor_ix = X_all[sc.neighbor_mask].index.tolist()
X_all['cluster_id'] = sc.cluster.labels_

### read data within neighborhood

In [43]:
sc.multi_events = X_all[sc.neighbor_mask]
sc.multi_events

     epochtime event  member_id  offset  counter  member_idx  cluster_id
2   1438907663  down          1      -1        1           0           2
3   1438907633  down          1      -1        1           0           2
4   1438907693  down          1      -1        1           0           2
5   1438907694    up          1      -1        1           0           2
9   1538907663    up          2      60        1           1           1
10  1538907633    up          2      60        1           1           1
11  1538907693  down          2      60        1           1           1
12  1538907695    up          2      60        1           1           1
16  1438907664  down          3     121        1           2           0
17  1438907634  down          3     121        1           2           0
18  1438907694    up          3     121        1           2           0
19  1438907694    up          3     121        1           2           0

## process multi events: majority wins

* remove redundant change_plan events foreach member
* keep the latest

In [44]:
# pick from where to read the result
# sc = spatial cluster
# nn = nearest neighbor
pme = sc

In [45]:
agg = pme.multi_events.groupby(['member_id', 'cluster_id', 'event'], as_index=False).agg({'counter': np.sum,
                                                                                          'epochtime': np.max})
agg

   member_id  cluster_id event  counter   epochtime
0          1           2  down        3  1438907693
1          1           2    up        1  1438907694
2          2           1  down        1  1538907693
3          2           1    up        3  1538907695
4          3           0  down        2  1438907664
5          3           0    up        2  1438907694

In [46]:
def majority_rule(x):
    if len(x[x == x.max()]) == 1:
        # max is unique
        return x.max()
    else:
        # tie-break
        x.ix[x.argmax()] -= 1
        return x

agg.groupby(['member_id', 'cluster_id'])['counter'].transform(majority_rule)

0    3
1    3
2    3
3    3
4    1
5    2
Name: counter, dtype: int64

In [47]:
agg[agg['counter'] == agg.groupby(['member_id', 'cluster_id'])['counter'].transform(majority_rule)]

   member_id  cluster_id event  counter   epochtime
0          1           2  down        3  1438907693
3          2           1    up        3  1538907695
5          3           0    up        2  1438907694

In [48]:
agg[agg['counter'] == agg.groupby(['member_id', 'cluster_id']).transform(max)['counter']]

   member_id  cluster_id event  counter   epochtime
0          1           2  down        3  1438907693
3          2           1    up        3  1538907695
4          3           0  down        2  1438907664
5          3           0    up        2  1438907694

### read data for the "noise"

* read the single change_plan from X

In [49]:
single_ix = np.delete(X_all.index, pme.neighbor_ix, None)
single_ix

Int64Index([0, 1, 6, 7, 8, 13, 14, 15, 20, 21], dtype='int64')

In [50]:
X_all.iloc[single_ix]

     epochtime event  member_id  offset  counter  member_idx  cluster_id
0   1438797641    up          1      -1        1           0          -1
1   1438807651  down          1      -1        1           0          -1
6   1439797999    up          1      -1        1           0          -1
7   1438797651    up          2      60        1           1          -1
8   1538807651  down          2      60        1           1          -1
13  1539797999  down          2      60        1           1          -1
14  1438797644    up          3     121        1           2          -1
15  1438807654    up          3     121        1           2          -1
20  1439797994  down          3     121        1           2          -1
21  1438907665    up          4     182        1           3          -1